# Finding base station neighbours via the Delaunay triangulation

We will reuse the work of Delphine PAQUIRY to find base stations' neighbours.

In [ ]:
# Importation of libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

from useful_methods.plots import plot_delaunay, plot_graph
from useful_methods.criterias import distance_criteria, quadrant_criteria, angle_criteria

In [ ]:
# Importing database
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df.head()

## Useful methods



### Creation of a graph based on delaunay triangulation

In [ ]:
import networkx as nx
from itertools import combinations

def delaunay_to_graph(delaunay_triangulation):
    """ Returns a Networkx Graph based on the Delaunay triangulation and the position of each node.
        
        Parameters
        ----------
        delaunay_triangulation : Delaunay
            Result of the Delaunay triangulation.

        Returns
        -------
        G : Graph
            A Networkx Graph graph.
        pos : dict
            The position of G's nodes.
    """
    G=nx.Graph()
    nodes = range(len(delaunay_triangulation.points))
    G.add_nodes_from(nodes) # adds nodes names (0 to number_of_points-1)

    for simplex in delaunay_triangulation.simplices:
        G.add_edges_from(combinations(simplex, 2))

    pos = dict(zip(nodes,delaunay_triangulation.points)) # gives each node his own position

    return G, pos

## First try on the Gard county

### Creation of the dataframe

In [ ]:
department = "Gard"

In [ ]:
df_gard = df.loc[df['nom_dep'] == department]
df_gard = df_gard.drop(columns=["code_op", "id_site_partage", "id_station_anfr", "nom_reg", "nom_dep", "insee_dep", "insee_com", "site_2g", "site_3g", "site_4g", "site_5g", "mes_4g_trim", "site_ZB", "site_DCC", "site_strategique", "site_capa_240mbps", "date_ouverturecommerciale_5g", "site_5g_700_m_hz", "site_5g_800_m_hz", "site_5g_1800_m_hz", "site_5g_2100_m_hz", "site_5g_3500_m_hz"])
df_gard.head()

### Delaunay triangulation on Free Mobile stations

In [ ]:
# Selecting the provider : Free Mobile
df_gard_free = df_gard.loc[df['nom_op'] == "Free Mobile"]
df_gard_free = df_gard_free.drop(columns=['nom_op'])
df_gard_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_gard_free_points = np.array(df_gard_free[['longitude', 'latitude']])

In [ ]:
delaunay_gard_free = Delaunay(df_gard_free_points)

In [ ]:
# plot_delaunay(delaunay_gard_free)

### Creating a graph based on delaunay triangulation

In [ ]:
G, pos = delaunay_to_graph(delaunay_gard_free)

In [ ]:
# plot_graph(G,pos)

### Addition of criterias

In [ ]:
G_dist = distance_criteria(G,pos)

In [ ]:
G_quad = quadrant_criteria(G,pos)

In [ ]:
G_angle = angle_criteria(G,pos)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(22,5))
# plot_delaunay(delaunay_gard_free,ax=axs[0],show=False)
plot_graph(G_dist,pos,ax=axs[2],show=False, title='Distance criteria')
plot_graph(G_quad,pos,ax=axs[0],show=False, title='Quadrant criteria')
plot_graph(G_angle,pos,ax=axs[1],show=False, title='Angle Delaunay')
fig.suptitle("Comparison of the Quadrant and Distance cirteria", fontsize=18, va='center')
plt.show()

### Comparison of the results

In [ ]:
G_modif = distance_criteria(G,pos)

In [ ]:
G_modif = quadrant_criteria(G_modif,pos)

In [ ]:
G_modif = angle_criteria(G_modif,pos)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,5))
plot_delaunay(delaunay_gard_free,ax=axs[0],show=False,department=department)
plot_graph(G_modif,pos,ax=axs[1],show=False,title="Delaunay after reduction",department=department)
fig.suptitle("Comparison of the Delaunay Triangulation and its reduction of edges", fontsize=18, va='center')
plt.show()

## Big Delaunay on France
Only Free Mobile

In [ ]:
# Selecting the provider : Free Mobile
df_free = df.loc[df['nom_op'] == "Free Mobile"]
df_free = df_free.drop(columns=['nom_op'])
df_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_free_points = np.array(df_free[['longitude', 'latitude']])

In [ ]:
delaunay_free = Delaunay(df_free_points)

In [ ]:
plot_delaunay(delaunay_free)

In [ ]:
G_france, pos_france = delaunay_to_graph(delaunay_free)

In [ ]:
G_france_modif = distance_criteria(G_france,pos_france)

In [ ]:
G_france_modif = quadrant_criteria(G_france_modif,pos_france)

In [ ]:
# G_france_modif = angle_criteria(G_france_modif,pos_france)

# /!\ beug ici

In [ ]:
# plot_graph(G_france_modif,pos_france, department=df['nom_dep'].unique())